# Mobile App for Lottery Addiction

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. The data set has data for 3,665 drawings, dating from 1982 to 2018 (we'll come back to this).

In [1]:
import pandas as pd
import numpy as np

In [4]:
# write two functions to calculate factorials and combinations
def factorial(n):
    fact = 1
    for i in range(1, n+1):
        fact *= i
    return fact

def combinations(n, k):
    return ( factorial(n) / (factorial(k) * factorial(n-k)) )

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

Inside the app, the user inputs six different numbers from 1 to 49.
Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [36]:
def one_ticket_probability(list_six):
    tot_outcomes = combinations(49, 6)
    probab_1_ticket = 1/tot_outcomes
    print('Your probability of winning the lottery with a {ticket} ticket is {perc:.7f}%'.format(ticket=list_six, perc=probab_1_ticket*100))

In [37]:
print(one_ticket_probability([1,22,3,4,5,6]))

Your probability of winning the lottery with a [1, 22, 3, 4, 5, 6] ticket is 0.0000072%
None


## Analysing the historical lottery data

We wrote a function that can tell users what is the probability of winning the big prize with a single ticket. For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

On this screen, we'll focus on exploring the historical data coming from the Canada 6/49 lottery. The data set can be downloaded from [Kaggle](https://www.kaggle.com/datascienceai/lottery-dataset).

In [38]:
lotto = pd.read_csv("649.csv")
lotto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT          3665 non-null   int64 
 1   DRAW NUMBER      3665 non-null   int64 
 2   SEQUENCE NUMBER  3665 non-null   int64 
 3   DRAW DATE        3665 non-null   object
 4   NUMBER DRAWN 1   3665 non-null   int64 
 5   NUMBER DRAWN 2   3665 non-null   int64 
 6   NUMBER DRAWN 3   3665 non-null   int64 
 7   NUMBER DRAWN 4   3665 non-null   int64 
 8   NUMBER DRAWN 5   3665 non-null   int64 
 9   NUMBER DRAWN 6   3665 non-null   int64 
 10  BONUS NUMBER     3665 non-null   int64 
dtypes: int64(10), object(1)
memory usage: 315.1+ KB


In [39]:
lotto.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [40]:
lotto.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


Now we're going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The engineering team told us that we need to be aware of the following details:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list and serve as an input to our function.
- The engineering team wants us to write a function that prints:
    - the number of times the combination selected occurred in the Canada data set; and
    - the probability of winning the big prize in the next drawing with that combination.

In [51]:
def extract_numbers(row):
    row = row[4:10]
    return set(row.values)

In [56]:
winning_numbers = lotto.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [59]:
def check_historical_occurrence(ticket, old_wins):
    ticket = set(ticket)
    check = old_wins == ticket
    print("The {ticket} ticket has occurred {occur} times in the past".format(ticket = ticket, occur = sum(check)))
    print("Regardless of that, you have a 0.0000072% probability of winning the lottery with your ticket.")

In [60]:
check_historical_occurrence([1,2,3,4,5,6], winning_numbers)

The {1, 2, 3, 4, 5, 6} ticket has occurred 0 times in the past
Regardless of that, you have a 0.0000072% probability of winning the lottery with your ticket.


Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning. Next, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

In [64]:
def multi_ticket_probability(num_tickets):
    tot_outcomes = combinations(49, 6)
    probab_win = num_tickets / tot_outcomes
    print("With {num_tickets} tickets, you have a {perc:.6f}% probability of winning the lottery.".format(num_tickets = num_tickets, 
                                                                                                     perc = probab_win*100))

In [65]:
for i in [1, 10, 100, 10000, 1000000, 6991908, 13983816]:
    multi_ticket_probability(i)

With 1 tickets, you have a 0.000007% probability of winning the lottery.
With 10 tickets, you have a 0.000072% probability of winning the lottery.
With 100 tickets, you have a 0.000715% probability of winning the lottery.
With 10000 tickets, you have a 0.071511% probability of winning the lottery.
With 1000000 tickets, you have a 7.151124% probability of winning the lottery.
With 6991908 tickets, you have a 50.000000% probability of winning the lottery.
With 13983816 tickets, you have a 100.000000% probability of winning the lottery.


Next, we're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

Inside the app, the user inputs:
- six different numbers from 1 to 49; and
- an integer between 2 and 5 that represents the number of winning numbers expected

Our function prints information about the probability of having the inputted number of winning numbers.

In [66]:
def probability_less_6(n_winning_numbers):
    n_combinations_ticket = combinations(6, n_winning_numbers)
    n_combinations_remaining = combinations(43, 6 - n_winning_numbers)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining
    
    n_combinations_total = combinations(49, 6)    
    probability = successful_outcomes / n_combinations_total
    
    probability_percentage = probability * 100    
    combinations_simplified = round(n_combinations_total/successful_outcomes)    
    print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_winning_numbers, probability_percentage,
                                                               int(combinations_simplified)))

In [68]:
for i in range(2,6):
    probability_less_6(i)
    print("---------------")

Your chances of having 2 winning numbers with this ticket are 13.237803%.
In other words, you have a 1 in 8 chances to win.
---------------
Your chances of having 3 winning numbers with this ticket are 1.765040%.
In other words, you have a 1 in 57 chances to win.
---------------
Your chances of having 4 winning numbers with this ticket are 0.096862%.
In other words, you have a 1 in 1,032 chances to win.
---------------
Your chances of having 5 winning numbers with this ticket are 0.001845%.
In other words, you have a 1 in 54,201 chances to win.
---------------
